# Modeling

## Carga de datos

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# Load the dataset
df_curated_stocks = pd.read_csv('../data/curated/df_curated_stocks.csv')

In [ ]:
df_curated_stocks.info(show_counts=True)

In [ ]:
df_curated_stocks.head()

In [ ]:
# Quitar nans
df_curated_stocks.dropna(inplace=True)

# Verificar
df_curated_stocks.info(show_counts=True)

## Clasificacion

La predicción de si una acción sube o baja es un problema de clasificación, ya que el objetivo es asignar una etiqueta discreta (por ejemplo, 1 si sube y 0 si baja). Por lo que se probaran algunos modelos para analizar que tan bien se ajustan al probelma. Por ejemplo se probaran:  
- Regresión Logística, 
- Árboles de Decisión 
- Random Forest 
- Ensambladores de Boosting 

In [ ]:
# Contar el total de acciones subiendo y bajando
df_curated_stocks['Accion_Sube'].value_counts()

In [ ]:
# Porcentaje
480569/1392123  

In [ ]:
# Split dataset (train y test)
train, test = train_test_split(df_curated_stocks, test_size=0.20, random_state=12345)

In [ ]:
# Quitar la columna que quiero predecir de ambos sets
train_caracteristica = train.drop(['Accion_Sube', 'Symbol'], axis=1)
train_objetivo = train['Accion_Sube']

test_caracteristica = test.drop(['Accion_Sube', 'Symbol'], axis=1)
test_objetivo = test['Accion_Sube']


In [ ]:
train_objetivo


### Regresion Logistica

In [ ]:
# Regresion Logistica
reg_log = LogisticRegression(random_state=12345)

In [ ]:
# Entrenamiento del modelo
reg_log.fit(train_caracteristica, train_objetivo)

In [ ]:
reg_log.predict(train_caracteristica)


In [ ]:
prediccion_train = reg_log.predict(train_caracteristica)

for valor in prediccion_train:
    if valor == 0:
        print('La accion bajo')
    elif valor == 1:
        print('La accion subio')


In [ ]:
prediccion_test = reg_log.predict(test_caracteristica)

for valor in prediccion_test:
    if valor == 0:
        print('La accion bajo')
    elif valor == 1:
        print('La accion subio')

In [ ]:
# Datos de entrenamiento
reg_log.score(train_caracteristica, train_objetivo)

In [ ]:
# Datos de test
reg_log.score(test_caracteristica, test_objetivo)

**Precision**

El modelo tiene una precision de 0.9987 para los datos de entrenamiento

El modelo tiene una precision de 0.9986 para los datos de prueba

**Clasificacion:**

- Sube la accion: 1
- Baja la accion: 0

Buenos: VP - VN

Malos: FP - FN

VP: El modelo predice que sube la accion -> y en la realidad sube la accion
VN: El modelo predice que baja la aacion -> y en la realidad baja la accion
FP: El modelo predice que sube la accion -> y en la realidad baja la accion
FN: El modelo predice que baja la accion -> y en la realidad sube la accion

### Arboles de decision

In [ ]:
arbol_clf = DecisionTreeClassifier()

In [ ]:
arbol_clf.fit(train_caracteristica, train_objetivo)

In [ ]:
arbol_clf.score(train_caracteristica, train_objetivo)

In [ ]:
arbol_predict_entrenamiento = arbol_clf.predict(train_caracteristica)

In [ ]:
for valor in arbol_predict_entrenamiento:
    if valor == 0:
        print('Predice que baja la accion')
    elif valor == 1:
        print('Predice que sube la accion')

In [ ]:
reg_log.coef_

In [ ]:
reg_log.intercept_

### Random Forest

### Ensambladores de boosting